<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/Hyper%20Parameter%20Tuning/Churn_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
@Author: Satwik Ram K

Churn Modeling Dataset

"""
# connecting to Kaggale
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
! kaggle datasets download -d shrutimechlearn/churn-modelling

  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 31.2MB/s]


In [3]:
! unzip /content/churn-modelling.zip

Archive:  /content/churn-modelling.zip
  inflating: Churn_Modelling.csv     


In [4]:
# Importing Librarires
import numpy as np
import pandas as pd


In [17]:
# Importing dataset
dataset = pd.read_csv('/content/Churn_Modelling.csv')

In [18]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [20]:
# Dropping unwanted columns
dataset.drop(columns = ['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)

In [21]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [22]:
dataset.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [23]:
dataset.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [24]:
columns = ['Geography', 'Gender']

In [25]:
for column in columns:
  dummy = pd.get_dummies(dataset[column], drop_first = True)
  dataset.drop(column, axis = 1, inplace = True)
  dataset = pd.concat([dummy, dataset], axis = 1)



In [26]:
dataset.head()

,Male,Germany,Spain,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,0,619,42,2,0.00,1,1,1,101348.88,1
1,0,0,1,608,41,1,83807.86,1,0,1,112542.58,0
2,0,0,0,502,42,8,159660.80,3,1,0,113931.57,1
3,0,0,0,699,39,1,0.00,2,0,0,93826.63,0
4,0,0,1,850,43,2,125510.82,1,1,1,79084.10,0


In [31]:
print(len(dataset.columns))

12


Taking X and Y

In [37]:
X = dataset.drop('Exited', axis = 1)
y = dataset['Exited']

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

Splitting the dataset

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 388)

Normalizing the dataset

In [40]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Building the model



In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [29]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 2.2MB/s 


building model without hyper parameter tuning

In [42]:
def build_model():
  model = Sequential()

  model.add(Dense(units = 512, activation = 'relu', input_dim = 11 ))

  model.add(Dense(units = 1024, activation = 'relu'))

  model.add(Dense(units = 1, activation = 'sigmoid'))

  return model


In [43]:
model = build_model()

In [44]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [46]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop = EarlyStopping(monitor = 'val_loss', patience = 10,
                          restore_best_weights = True,
                          min_delta = 0, verbose = 1)

checkpoint = ModelCheckpoint('churn.h5',
                             monitor = 'val_loss', save_best_only = True,
                             mode = 'min', verbose = 1)

callbacks = [earlystop, checkpoint]

Training the model

In [47]:
model.fit(X_train, y_train, validation_split = 0.2, verbose = 1, epochs = 100, callbacks = callbacks)

Epoch 1/100
198/200 [============================>.] - ETA: 0s - loss: 0.1117 - accuracy: 0.9564
Epoch 00001: val_loss improved from inf to 0.60381, saving model to churn.h5
200/200 [==============================] - 2s 12ms/step - loss: 0.1122 - accuracy: 0.9558 - val_loss: 0.6038 - val_accuracy: 0.8300
Epoch 2/100
200/200 [==============================] - ETA: 0s - loss: 0.1149 - accuracy: 0.9538
Epoch 00002: val_loss did not improve from 0.60381
200/200 [==============================] - 3s 13ms/step - loss: 0.1149 - accuracy: 0.9538 - val_loss: 0.6327 - val_accuracy: 0.8250
Epoch 3/100
199/200 [============================>.] - ETA: 0s - loss: 0.1061 - accuracy: 0.9570
Epoch 00003: val_loss did not improve from 0.60381
200/200 [==============================] - 2s 12ms/step - loss: 0.1062 - accuracy: 0.9570 - val_loss: 0.6508 - val_accuracy: 0.8275
Epoch 4/100
200/200 [==============================] - ETA: 0s - loss: 0.0971 - accuracy: 0.9628
Epoch 00004: val_loss did not improve

Almost 98% Accuracy and stopped at 11 epochs

In [48]:
from tensorflow.keras.models import load_model

In [49]:
loaded_model = load_model('/content/churn.h5')

In [52]:
loss, score = loaded_model.evaluate(X_test, y_test, verbose = 1)

63/63 [==============================] - 0s 3ms/step - loss: 0.6582 - accuracy: 0.8205


In [53]:
score*100

82.05000162124634

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [55]:
y_pred = loaded_model.predict(X_test)

In [59]:
y_pred = (y_pred > 0.5)

In [60]:
acc = accuracy_score(y_test, y_pred)

In [61]:
acc * 100

82.05

Let us improve the model using hyper Parameter tuning

Building model

In [90]:
def hyper_model(hp):
  model = Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units' , min_value = 32, max_value = 512, step = 32)

  hp_kernal = hp.Choice('kernel_initializer' , values= ['uniform', 'glorot_uniform'])


  model.add(Dense(units = hp_units, kernel_initializer = hp_kernal , activation = 'relu', input_dim = 11))

  hp_units1 = hp.Int('units' , min_value = 256, max_value = 1024, step = 32)

  model.add(Dense(units = hp_units1,kernel_initializer = hp_kernal, activation = 'relu'))

  hp_units2 = hp.Int('units' , min_value =1024 , max_value = 2048, step = 32)

  model.add(Dense(units = hp_units2, kernel_initializer = hp_kernal, activation = 'relu'))

  model.add(Dense(1, activation = 'sigmoid'))

  hp_optimizer = hp.Choice('optimizer',  values = ['adam', 'RMSProp'])

  model.compile(optimizer = hp_optimizer,
                loss = 'binary_crossentropy', 
                metrics = ['accuracy'])
  
  return model



Instantiate the tuner and perform hypertuning

In [92]:
tuner = kt.Hyperband(hyper_model,
                     objective = 'val_accuracy', 
                     max_epochs = 100,
                     factor = 3,
                     directory = 'Tuned_model',
                     project_name = 'Tuned_Churnmodel')

In [93]:
import IPython

In [94]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [95]:
tuner.search(X_train, y_train, epochs = 100, validation_split = 0.2, callbacks = [ClearTrainingOutput()])

INFO:tensorflow:Oracle triggered exit


In [96]:

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [97]:
# Build the model with the optimal hyperparameters and train it on the data
tuned_model = tuner.hypermodel.build(best_hps)
tuned_model.fit(X_train, y_train, epochs = 100, validation_split = 0.2,  callbacks = callbacks, verbose = 1)

Epoch 1/100
189/200 [===========================>..] - ETA: 0s - loss: 0.4183 - accuracy: 0.8266
Epoch 00001: val_loss improved from 0.60381 to 0.35150, saving model to churn.h5
200/200 [==============================] - 1s 3ms/step - loss: 0.4148 - accuracy: 0.8280 - val_loss: 0.3515 - val_accuracy: 0.8650
Epoch 2/100
183/200 [==========================>...] - ETA: 0s - loss: 0.3481 - accuracy: 0.8538
Epoch 00002: val_loss improved from 0.35150 to 0.33819, saving model to churn.h5
200/200 [==============================] - 1s 3ms/step - loss: 0.3506 - accuracy: 0.8533 - val_loss: 0.3382 - val_accuracy: 0.8669
Epoch 3/100
198/200 [============================>.] - ETA: 0s - loss: 0.3416 - accuracy: 0.8583
Epoch 00003: val_loss did not improve from 0.33819
200/200 [==============================] - 1s 3ms/step - loss: 0.3416 - accuracy: 0.8580 - val_loss: 0.3435 - val_accuracy: 0.8631
Epoch 4/100
197/200 [============================>.] - ETA: 0s - loss: 0.3331 - accuracy: 0.8610
Epoch 

In [99]:
loss1, acc1 = tuned_model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 2ms/step - loss: 0.3503 - accuracy: 0.8610


In [100]:
print("After tuning the accuracy is:",acc1 * 100,"%")

After tuning the accuracy is: 86.10000014305115 %


Still can be tuned by changing the architecture!!